# TPM034A Machine Learning for socio-technical systems 
## `Lab session 03:  Decision trees and Random forests`

**Delft University of Technology**<br>
**Q2 2022**<br>
**Instructor:** Sander van Cranenburgh <br>
**TAs:**  Francisco Garrido Valenzuela & Lucas Spierenburg <br>

### `Instructions`

**Lab sessions aim to:**<br>
* Show and reinforce how models and ideas presented in class are used in practice.<br>
* Help you gather hands-on machine learning skills.<br>

**Lab sessions are:**<br>
* Learning environments where you work with Jupyter notebooks and where you can get support from TAs and fellow students.<br> 
* Not graded and do not have to be submitted. 

### `Workspace set-up`
**Option 1: Google Colab**<br>
Uncomment the following cells code lines if you are running this notebook on Colab

In [ ]:
#!git clone https://github.com/TPM34A/Q2_2022
#!pip install -r Q2_2022/requirements_colab.txt
#!mv "/content/Q2_2022/Lab_sessions/lab_session_03/data" /content/data

**Option 2: Local environment**<br>
Uncomment the following cell if you are running this notebook on your local environment. This will install all dependencies on your Python version.

In [ ]:
#!pip install -r requirements.txt

### `Application: Predicting primary school outflow` <br>
In this lab session we will use decision trees and random forests to predict the **share of school leavers advised to go to higher education (HAVO, VWO)**. In the last year of primary education, schools give their students an advice on their further education. Data on the school advices are collected by the ministry of education to monitor school outflow. These data are also widely used by parents who use them to help choose the primary school for their offspring. Various websites have recently emerged that make these school data easily accessible, see e.g. the screenshot below taken from [allecijfers.nl](https://allecijfers.nl/basisschool/freinetschool-delft/) showing the stats for de Freinet school in Delft.<br>
<br>
![website_reporting_outflow](website_screenshot.jpg)
<br>
<br>
In this lab session we are going to analyse these data. In particular, our **aim** is to answer the following 2 Research Questions (RQs) that parents may consider upon choosing a school for their offspring:
* RQ1: Is the share of school leavers with an advice for higher education, larger at **large schools** than at **small schools**, or vice versa?
* RQ2: Does the type of school (i.e its **denomination**) matters for the share of school leavers with an advice for higher education?<br> 

**Background information on the Dutch school system**<br>
The majority of the schools in the Netherlands are grounded in specific religious or philosophical beliefs, such as Christianity, Protestantism and Anthroposophy. This is called school denomination. Schools that are not grounded in such belief are called "Openbare scholen", meaning "Public schools". Importantly, virtually all schools in the Netherlands are funded by the federal government. Thus, unlike their counterparts in various Anglo-Saxon countries, public schools are generally equally well-funded as schools with specific denominations. 
<br>

**Neighboorhood characteristics**<br>
In the literature, there is ample evidence that neighboorhood characteristics correlate with educational achievements [see e.g. Owens 2018](https://journals.sagepub.com/doi/full/10.1177/0038040717741180). Accordingly, these characteristics can be expected to correlate with our target feature: the share of school leavers with an advice for higher education.<br>

The **conceptual model** for this lab session is shown below. Accordingly, to do the proposed analyses we have pulled together data on:
* School characteristics. These data include the size and denomination of schools (`TOTAL_STUDENTS`, `DENOMINATION`) as well as the share of the school leavers with an educational advice of HAVO or Higher (`SHARE_HIGH`)
* Neighbourhood characteristics. Data on the neighbourhoods in which the schools are located (population density, demography, real-estate value...).

![conceptual_model_lab_ session3](Conceptual_model_lab_session3.png)

<br>

**Learning objectives**. After completing the following exercises you will be able to: <br>
1. Apply multiple ML models to the same data set, including `Linear multiple regressions`, `Decision trees` and `Random Forests`
2. Identify the most important features and their impact on the target feature<br>
3. Explore the pros and cons of each model, and how models can complement each other to answer specific research questions <br>


#### `Organisation`
This lab session comprises **`3`** parts:
1. Loading, exploring and cleaning the data
2. Training multiple models: `Linear multiple regressions`, `Decision trees` and `Random Forests`
3. Comparing and reflecting on the model performances and their outcomes

and comprises **`6`** `exercises`.

In [ ]:
# Import required Python packages and modules
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from os import getcwd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV,cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,VotingRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer,log_loss
from sklearn import tree
from sklearn.preprocessing import StandardScaler

# Setting
pd.set_option('display.max_columns', None)

#### **1. Loading, exploring and cleaning the data**

In [ ]:
# Load the data into a pandas DataFrame
data_folder = Path('data')
df_schools = pd.read_csv(data_folder/'school_data.csv')

Before creating prediction models, we have to understand what features are in your data.<br><br>
Let's start with inspecting the **target feature** `SHARE_HIGH`<br>
`SHARE_HIGH` is the share of the students that have received the advice for further education of HAVO, or higher.

In [ ]:
df_schools['SHARE_HIGH'].hist()

From this histogram two observations can be made:
1. The target feature `SHARE_HIGH` is by and large normally distributed.
2. What catches the eye though, is the uptick around zero. 

Next, let's inspect the first (explanatory) feature of interest, i.e. `TOTAL_STUDENTS`.

In [ ]:
df_schools['TOTAL_STUDENTS'].hist()

In [ ]:
# Some buurt features depend on their size (e.g. the number of people under 15 years old for instance).
# We divide these feature by the total number of inhabitants in the neighbourhood (buurt): thus, `number_inhabitants_5`.

# Features to divide by the buurt population.
f2divide = ['Men_6', 'Women_7', 'k_0To15Years_8', 'k_15To25Years_9',
            'k_25To45Years_10', 'k_45To65Years_11', 'k_65YearsOrder_12',
            'Single_13', 'Married_14', 'Divorced_15', 'Widowed_16',
            'WesternTotal_17', 'Non-WesternTotal_18', 'Morocco_19',
            'DutchAntillesAndAruba_20', 'Suriname_21', 'Turkey_22',
            'OtherNon-Western_23', 'HouseholdsTotal_28', 'Single_Households_29',
            'HouseholdsWithoutChildren_30', 'HouseholdsWithChildren_31',
            'Housing_Stock_34','Motorcycles_104','PassengerCarTotal_99']

df_schools.loc[:,f2divide] = df_schools.loc[:,f2divide].div(df_schools['number_inhabitants_5'], axis = 0)

# Some other features are present both in absolute and relative terms. 
# In that case, we keep only the relative terms.
f2remove = ['BirthTotal_24','MortalityTotal_26','EducationLevelLow_64', 'EducationLevelMedium_65',
       'EducationLevelHigh_66','PassengerCarFuelBenzine_100', 'PassengerCarSOverigeFuel_101']

df_schools = df_schools.drop(columns = f2remove)



### <span style="color:skyblue">Exercise 1: Do school with more than the average number of students have on average higher advices?</span> 
To get a first hunch regarding the first research question:<br>
`A` Create a new column called `SIZE` in the dataframe, consisting of two groups (0,1):<br> 
    0 --> `TOTAL_STUDENTS` smaller than the mean number of students<br>
    1 --> `TOTAL_STUDENTS` larger than the mean number of students<br>
`B` Put the results in a boxplot, where *y* is the target feature (SHARE_HIGH). <br>
`C` What is your first hunch: do large schools perform 'better'? <br>
`D` Suppose that large schools do perform better, what could be a driving force the difference that we observe? <br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)
# ANSWERS




Next, we inspect the second feature of interest, i.e. the `DENOMINATION`.


In [ ]:
df_schools['DENOMINATION'].value_counts().plot(kind='bar')

As we can see in the histogram, the distribution of "DENOMINATION" is highly skewed. Therefore, for our later analyses, we merge the little occuring categories into one new category called "DENOMINATION_Other"

In [ ]:
# Replace little occuring denomination with the category 'Other

# Create a list with little occuring denominations 
df_schools_denom_count = df_schools['DENOMINATION'].value_counts()

# Identify the denominations to be pulled together
denomination_threshold = 100
denom2merge= list(df_schools_denom_count.index[df_schools_denom_count<denomination_threshold])

# Replace little occuring denomitations with Other
df_schools['DENOMINATION'].replace(denom2merge,'Other',inplace = True)

# Show the new categories and their counts
df_schools['DENOMINATION'].value_counts()

Let's investigate the spread and skewness of "SHARE_HIGH" across school with different "DENOMINATION_ESTABLISHMENT". Hence, we create a boxplot.

In [ ]:
fig, ax = plt.subplots(1,1,figsize =(15,4))
sns.boxplot(x='DENOMINATION', y='SHARE_HIGH', data=df_schools, ax= ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

The boxplot shows no particular trend betweem denomination and school performance (i.e. SHARE_HIGH) 

All the features that we want to us in our ML models must be either *floats* or *integers*. Hence, we must convert categorial features (strings) of interest into dummy coded features. The only categorical feature of interest is `DENOMINATION`. Other categorical features, such as `INSTITUTION_NAME_BRANCH`, `LOCALITY_NAME`, `PROVINCE`, and  `SPECIES_PO` do not seem to be of interest and not converted into dummies.

In [ ]:
cat_features = ['DENOMINATION']
df_schools_cat = pd.get_dummies(df_schools[cat_features], columns = cat_features)

# Concatenate the categorical data and the original df
df_schools = pd.concat([df_schools,df_schools_cat], axis =1)

# Show the first line of the data frame
df_schools.head(1)

To identify other urban or school related features that could help predict the target (SHARE_HIGH), we create a heatmap showing the correlations across the features.

In [ ]:
# heatmap of correlations
# Create plot
fig, axes = plt.subplots(figsize=(30, 30))
fig.set_tight_layout(True)

# Compute correlation matrix
corr = df_schools.corr()

# Create upper triangular matrix to mask the upper triangular part of the heatmap
corr_mask = np.triu(np.ones_like(corr, dtype=bool))

# Generate a custom diverging colormap (because it looks better)
corr_cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr, mask = corr_mask, cmap=corr_cmap, annot=False,square = True, linewidths=.5, ax = axes)
plt.show()

We are specifically interested in the features that correlate with `HIGH_SHARE`. If a feature does not correlate with the target, it is unlikely it will help the model predict the target. Therefore, we create a list with 30 features that correlate strongest with the target feature. These features will be used in to train the ML models.

In [ ]:
# Compute absolute correlations with the target 
df_corr = corr[['SHARE_HIGH']].abs().sort_values(by=['SHARE_HIGH'],ascending=False).T

# Sort the features based the the strength of the correlation
features_corr_sorted  = list(df_corr.keys())

# Create list with the 30 most strongly correlated features, (we start at the index 1 to exclude the target itself)
features_corr = features_corr_sorted[1:31]
print(f'Features with the strongest correlation with the target:\n {features_corr}')

Regardless of whether e.g. `DENOMINATION` and `TOTAL_STUDENTS` correlate with the target, we must include them in our list of features. TOTAL_STUDENTS is already in the list of features that strongly correlate with the target. So, we only have to add the list of features of the denominations.

In [ ]:
features = features_corr + list(df_schools_cat.keys())
print(f'Total list of features:\n {features}')

Let's check the ranges of our features. This is relevant to know when interpreting the results of the regression model.

In [ ]:
df_schools[features].describe()

Now we can create the train and test data sets, which we will use to train our ML models.

In [ ]:
# Create train and test data 

# Create a df with the features only 
X = df_schools[features]

# Target
Y = df_schools.loc[:,'SHARE_HIGH']

# Split the data into a train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=12)

#### **2. Training multiple models: Regression model, Decision tree, and a Random forest**<br>
Often, a researcher does not know beforehand which sort of model will do well, and which will not. Therefore, ML researchers often apply multiple models to their task and pull together their outcomes.<br>
Next, we are going to apply 3 models:
1. Regression model (benchmark model)<br>
2. Decision tree<br>
3. Random Forest<br>

As we will see, each of these models provide complementary insights.

Because we are going to compare different models, we create a custom evaluation function that allows us to swiftly report the following stats for the train and test data:
* mean square error
* mean absolute error
* R2

In [ ]:
def eval_regression_perf(model, X_train, X_test, Y_train, Y_test):
    
    # Make prediction with the trained model
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)

    # Create a function that computes the MSE, MAE, and R2
    def perfs(Y,Y_pred):
        mse = mean_squared_error(Y,Y_pred)
        mae = mean_absolute_error(Y,Y_pred)
        R2 = r2_score(Y,Y_pred)
        return mse,mae,R2

    # Apply the perfs function to the train and test data sets
    mse_train, mae_train, r2_train = perfs(Y_train,Y_pred_train)
    mse_test,  mae_test , r2_test  = perfs(Y_test,Y_pred_test)
        
    # Print results
    print('Performance')
    print(f'Mean Squared  Error Train | Test: \t{mse_train:>7.4f}\t|  {mse_test:>7.4f}')
    print(f'Mean Absolute Error Train | Test: \t{mae_train:>7.4f}\t|  {mae_test:>7.4f}')
    print(f'R2                  Train | Test: \t{ r2_train:>7.4f}\t|  {r2_test:>7.4f}\n')

##### 2.1. Linear multiple regression model
This model usually serves as a benchmark when creating ML models. Therefore, we start with this model.

In [ ]:
# Create and train a multiple linear regression model
regr = LinearRegression(fit_intercept = True)

# Fit the model on the training data
regr.fit(X_train,Y_train)

# Evaluate the performance of the trained model
eval_regression_perf(regr, X_train, X_test, Y_train, Y_test)

# We take an ML approach (i.e. we use a train and tests set). But, if we deem the assumptions one would take from a statistical approach plausible, 
# then the weights obtained here are also interpretable. Let's therefore print the weights of the trained model
print('Weights')
print(f'Intercept: \t\t\t\t\t\t {regr.intercept_:>7.4f}')
for n in range(len(regr.coef_)):
    print(f'Weight_{X.keys()[n]:32s} \t\t {regr.coef_[n]:>7.4f}')

Based on these results, we can make several inferences:
1. The model fit is not great but also not very bad (R2 of ~0.35)
2. The model does not seem to overfit the data: it attains the same performance on the test as on the train data sets
3. From the positive weight associated with TOTAL_STUDENTS, we tentatively conclude that larger schools result in on avg higher share of advices for higher education.
4. From the weights associated with school denomination, we tentatively conclude that some denominations might impact on the share of advices for higher education.
5. From the remaining weights, we infer that their signs are mostly in line with intuition.


##### 2.2. Decision tree
The next ML model we will try on these data is the decision tree.

In [ ]:
# Create and train a decision tree
max_depth =4
max_leaf_nodes = 8
min_samples_leaf = 100
dt = DecisionTreeRegressor(criterion = "squared_error", max_depth=max_depth, max_leaf_nodes = max_leaf_nodes,min_samples_leaf=min_samples_leaf,random_state=5)

# Train the decision tree
dt.fit(X_train,Y_train)

# Evaluate the performance of the trained model
eval_regression_perf(dt, X_train, X_test, Y_train, Y_test)

# Visualise the decision tree
fig = plt.figure(figsize=(20,10))
tree.plot_tree(dt, feature_names = features, filled=True)
plt.show()

From the decision tree, we can make several observations:
1. Compared to the regression model, the model fit (in terms of e.g. R2) has substantially improved
2. The decison tree does not seem to over fit: the performance of the train and test data are more or less the same.
3. The tree is easily interpretable. Moreover, the tree structure (splits) are in line with intuition.

### <span style="color:skyblue">Exercise 2: Manually navigate through the decision tree to obtain a prediction for the first instance of X_test (X_test.loc[0])</span> 

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)
# ANSWERS

Let's see which features are found to be most important to our Decision tree.

In [ ]:
fig, ax = plt.subplots(figsize=(8,12))
sns.barplot(y=features,x=dt.feature_importances_, ax = ax).set(title='Feature importance to predict share of school outflow that is high')
ax.set_xlabel('Feature importance')

# Print most important features
sorted_indices = np.argsort(dt.feature_importances_)[::-1]
most_imp_features_dt = [features[i] for i in sorted_indices]
print(f'Top 5 most important features:\n {most_imp_features_dt[:5]}')

# Show the plot
plt.show()

From the importance plot, we can draw several conclusions:
1. Most features seem not to help predict the target: many features attain a (nearly) zero feature_importances
2. Just four features seem important to predict the target: 'TOTAL_STUDENTS', 'pctEducationLevelHigh', 'pctEducationLevelLow', 'AverageWOZValueOfHomes_35'. These feature are in line with expectations.
3. The school denomination seems unimportant in the predictions

### <span style="color:skyblue">Exercise 3: Try to improve the model fit of the decision tree, by testing different numbers for:</span> 
* max_depth <br>
* max_leaf_nodes<br>
* min_samples_leaf<br>

How does each of these hyperparameters affects:<br>
`A` The size of the tree<br>
`B` Model performance, in terms of R2. <br>
`C` Feature importance <br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)
# ANSWERS


##### 2.3. Random Forest
Lastly, we are going to try a RF on these data. RFs are known to attain a high model performance on a wide range of regression and classification tasks. 
<br>
Instead of manually searching for the optimal hyperparameters for the RF, which is laborious, let's use sk-learn's GridSearchCV functionality to automate the hyperparameter search.

In [ ]:
# Create RF object
rf_gs = RandomForestRegressor(n_estimators=250,criterion= "squared_error",random_state=5)

# Define the hyperparameter search space
hyperparameter_space = {
    'max_depth': [5,7, 9, 11],
    'max_leaf_nodes': [20,30],
    'min_samples_leaf': [50, 100],
    'max_features':[0.5,0.7,0.9]}

# Create scoring function
scorer = make_scorer(mean_squared_error, greater_is_better = False)

# Create the grid_search object, with using the MLP classifier
folds = 5 # Number of cross validation splits
rf_gridsearch = GridSearchCV(rf_gs, hyperparameter_space, n_jobs=-1, cv=folds, scoring=scorer, return_train_score=True)

In [ ]:
# Execute the training/gridsearch
# Note that this is computationally expensive! 
# It may take up to 5 minutes, since multiple models need to be trained multiple times
rf_gridsearch.fit(X_train,Y_train)
print(f'Optimal hyperparameters:\n{rf_gridsearch.best_params_}')
print(f'Mean Squared Error:\t{(-rf_gridsearch.best_score_):0.3f}') 

In [ ]:
# Set the hyperparameters of the Random Forest to the best values found, e.g. 
max_depth = 9
max_features = 0.9
max_leaf_nodes = 30
min_samples_leaf = 50

# Create the Random Forest object with the best hyperparameters
rf = RandomForestRegressor(criterion = "squared_error", n_estimators=1000, max_features=max_features, max_depth=max_depth, max_leaf_nodes = max_leaf_nodes,min_samples_leaf=min_samples_leaf,random_state=5)

# Train the Randon Forest
rf.fit(X_train,Y_train)

# Evaluate the performance of the hyperparameter optimised RF model
eval_regression_perf(rf,X_train,X_test, Y_train, Y_test)

In [ ]:
# To obtain an understanding of the complexity of the learned trees, we visualise one of the decision trees populating the forest
fig = plt.figure(figsize=(20,10))

# Tree id to visualise (integer between 0 and n_estimators). You can visualise different trees to assess their diversity
vis_tree = 0

# Create the tree plot
tree.plot_tree(rf.estimators_[vis_tree], feature_names = features, filled=True)

# Show the plot
plt.show()

Let's see which features are found to be most important to our Random Forest.

In [ ]:
# Plot the feature importances
fig, ax = plt.subplots(figsize=(8,10))
sns.barplot(y=features,x=rf.feature_importances_, ax = ax).set(title='Feature importance to predict share of school outflow that is "High"')
ax.set_xlabel('Feature importance')

# Print most important features
sorted_indices = np.argsort(rf.feature_importances_)[::-1]
most_imp_features_rf = [features[i] for i in sorted_indices]
print(f'Top 5 most important features:\n {most_imp_features_rf[:6]}')

# Show plot
plt.show()

### <span style="color:skyblue">Exercise 4: Compare the results of the Random forest with those of the Decision tree, in terms of fit, trees, and feature importance</span> 
`A` What difference do you see in terms of model performance?<br>
`B` What difference do you see in terms of the tree structure? (hint: visualise different trees from the forest to assess diversity) <br>
`C` What difference do you see in terms of the discovered feature performances? Specifically, also look at TOTAL_STUDENTS and DENOMINATION<br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)
# ANSWERS

#### **3. Comparing and reflecting on the model performances and their outcomes** 

Often it helps the researcher to pull together the outcomes of multiple models. The idea of doing this is closely related to wisdom of the crowd principle. If multiple different model point towards the same conclusion, then the researcher has compounding evidence. In contrast, if different models point towards different conclusions, then the researcher knows the conclusions are weak(er) and sensitive to the choice of model.<br>
Let's pull together the predictions of the 3 ML models, and show them next to one another.

In [ ]:
# Scatter true shares (x-axis) against the predicted shares by the different ML models (y-axis)
fig, ax = plt.subplots(1,3,figsize=(18,4))
ax[0].scatter(x = Y_test, y = regr.predict(X_test), s = 8, alpha = 0.8, c = '#ff7f0e', label = f'Regression | R2 = {regr.score(X_test,Y_test):0.3f}')
ax[1].scatter(x = Y_test, y = dt.predict(X_test),   s = 8, alpha = 0.8, c = '#2ca02c', label = f'Decision tree | R2 = {dt.score(X_test,Y_test):0.3f}')
ax[2].scatter(x = Y_test, y = rf.predict(X_test),   s = 8, alpha = 0.8, c = '#d62728', label = f'Random Forest | R2 = {rf.score(X_test,Y_test):0.3f}')

# Add labels, legend, and title to each plot
for n in range(0,3,1):
    ax[n].set_xlabel('True share')
    ax[n].set_ylabel('Predicted share')
    ax[n].set_xlim(0,1)
    ax[n].set_ylim(0,1)
    ax[n].legend()
    ax[n].set_title('True versus predicted SHARE_HIGH (test data)')

plt.show()

### <span style="color:skyblue">Exercise 5:  Comparison of predictions</span> 
`A` What is the most striking difference between the predictions across the three ML models? Can you explain this?<br>
`B` How would the (ideal) scatter plot look like? I.e. when the model would perfectly predict our target?


In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)


Another way the compare the predictions of the three models is by creating a **kernel density plot**. A kernel density plot visualises the distribution, i.e. the probability density, of a variable, in a smooth way, see e.g. [wikipedia.org/wiki/Kernel_density_estimation](https://en.wikipedia.org/wiki/Kernel_density_estimation). 

In [ ]:
# Plot kernel densities for the test data
fig, ax = plt.subplots(figsize=(10, 7))

# Plot kernel density of the true distribution, i.e of Y_test
sns.kdeplot(data = Y_test, ax=ax, legend=True,fill=True,alpha=.3, clip =(0,1), label = 'True')

# Plot kernel densities of the predicted distributions, based on the three models
sns.kdeplot(data = regr.predict(X_test),ax=ax, legend=True,fill=True,alpha=.3, clip =(0,1), label = f'Regression | R2 = {regr.score(X_test,Y_test):0.3f}')
sns.kdeplot(data = dt.predict(X_test),  ax=ax, legend=True,fill=True,alpha=.3, clip =(0,1), label = f'Decision tree | R2 = {dt.score(X_test,Y_test):0.3f}')
sns.kdeplot(data = rf.predict(X_test),  ax=ax, legend=True,fill=True,alpha=.3, clip =(0,1), label = f'Random Forest | R2 = {rf.score(X_test,Y_test):0.3f}')

# Add legend and title to the plot
#ax.legend(['True','Regression model', 'Decision tree','Random Forest'])
ax.legend()
plt.title('True vs predicted SHARE_HIGH')
ax.set_xlim(0,1)
plt.show()

#### **4. Complementarity of the models**
- Each model gives us different insights on the relationship explored.
- The linear regression model gives a quick overview of the associations between the variable we want to predict and the features we have.
- The linear regression mode cannot capture nonlinear relationships without intervention from the analyst: **the size of the school** has a very strong impact on the school performance, but this impact is non-linear, and the regression model underfits. 
- The decision tree can capture nonlinear relationships, and the analyst can easily interprete the model.
- However, the decision tree may overfit (for instance if the maximum tree depth is set to high).
- The random forest captures also nonlinear relationships better than the decision tree, but is somehwhat less straightforward to interprete. It reduces the variance of DTs.
- Hence, the first two models are useful in the exploratory analysis phase, while the random forest (and similarly flexible machine learning approaches) are especially useful to improve the predicting power.

##### 4.1. Model development cycle

If you have carefully inspected the trees produced by the DT and RF, you might have noticed the first split is (often) based on very small schools: TOTAL_STUDENTS < 11. When we go back to our data we see that this strongly correlates with a categorical feature called `SPECIES_PO` (see left-hand side boxplot below). Unaware of its meaning, at the start of the data exploration of this lab session we too quickly (and erroneaously) discarded this feature as irrelevant. `SPECIES_PO` has two categories: Bo and Sbo. 'Bo' stands for conventional education, while 'Sbo' stands for special education. Sbo schools are schools specifically targeted to cater for students that would not do well in the standard school system and are ussually small. A simple boxplot shows a strong effect of the `SPECIES_PO` on the target feature (see right-hand side plot below). This relationship totally makes sense. With this new insight we are able to improve the models, e.g. include `SPECIES_PO` specification of the linear regression model in a next cycle of the model development. 

In [ ]:
 # Boxplot SPECIES_PO vs SHARE_HIGH
fig, ax = plt.subplots(1,2,figsize =(12,5))
sns.boxplot(x='SPECIES_PO', y='TOTAL_STUDENTS', data=df_schools, ax= ax[0])
ax[0].set_xticklabels(ax[0].get_xticklabels(),rotation=90)

sns.boxplot(x='SPECIES_PO', y='SHARE_HIGH', data=df_schools, ax= ax[1])
ax[1].set_xticklabels(ax[1].get_xticklabels(),rotation=90)
plt.show()

### <span style="color:skyblue">Exercise 6:   Conclusion and reflection on the results (Discuss with the TAs)</span> 
In this lab session we have analysed the school data, and made predictions for SHARE_HIGH using three ML models.<br>
`A` Based on the above analyses, which of these models can best be used to predict SHARE_HIGH? Explain your answer.<br>
`B` In the kernel density plots, we see that the true variance of SHARE_HIGH (in blue) is considerably larger than the variance of SHARE_HIGH predicted by any of the ML models. What does this tell us about this regression problem?<br>
`C` Answer RQ1: Is the share of school leavers with an advice for higher education, larger at **large schools** than at **small schools**? Or is it impossible to say?<br>
`D` Answer RQ2: Does the type of school (i.e its **denomination**) matters for the share of school leavers with an advice for higher education?  Or is it impossible to say?<br>
`E` How has applying multiple ML models helped you to answer the research questions?<br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

##### **5. Ensemble models**
- Ensemble models are combinations of multiple models.
- In the cell below, we show how to implement an ensemble model.
- That ensemble model is composed of the regression model and decision tree.

In [ ]:
# Create ensemble objects
er = VotingRegressor([('lr', regr), ('dt', dt)])

# Fit ensemble objects
er.fit(X_train,Y_train)

# Evaluate the performance of the ensemble
print('Results ensemble')
eval_regression_perf(er,X_train, X_test,Y_train,Y_test)


**Observation:**<br>
* Since the regression model does not perform very well (i.e. without using the SPECIES_PO feature), the ensemble performs worse in this case than the individual DT.